In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log

%matplotlib inline

# GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = "/scratch/wenyuan/Mask_RCNN_On_Pathology/Mask_RCNN/mask_rcnn_coco.h5"

Using TensorFlow backend.


In [2]:
import prostate
dataset_dir = "/scratch/wenyuan/Mask_RCNN_On_Pathology/Data_Pre_Processing/cedars-224"
# held_out_set = 4
# mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
# class TrainConfig(prostate.ProstateConfig):
#             MEAN_PIXEL = np.array(mean_pixel)
# config = TrainConfig()
# config.display()

In [3]:
# subset_1 = [0, 3]
# dataset_train = prostate.ProstateDataset()
# dataset_train.load_prostate(dataset_dir, subset_1, mode = 16)
# dataset_train.prepare() # prepare the dataset class for use

In [4]:
# Create model in training mode
# model = modellib.ResNet_Classifier(mode="training", config=config,
#                           model_dir=MODEL_DIR)

In [5]:
# exclude = []
# if config.USE_TUMORCLASS:
#     tumor_head = ["tumor_class_conv1", "tumor_class_bn1", "tumor_class_conv2",
#                  "tumor_class_conv2", "tumor_class_bn2", "tumor_class_logits", "tumor_class"]
#     exclude = exclude + tumor_head

# model.load_weights(COCO_MODEL_PATH, by_name=True,
#                        exclude=exclude)

In [6]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
# model.train(dataset_train, dataset_train, 
#             learning_rate=config.LEARNING_RATE, 
#             epochs=5, 
#             layers='heads')

In [7]:
held_out_set = 4
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
class InferenceConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    DETECTION_NMS_THRESHOLD = 0.7     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

inference_config = InferenceConfig()
inference_config.display()
# Recreate the model in inference mode
model = modellib.ResNet_Classifier(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.7
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97800579  120.89113632  183.79060979]
MINI_MASK_SHAPE                (56, 56)
NAME                           prostate
NUM_CLASSES                    4
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
PO

In [8]:
model_path = model.find_last()[1]
# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /scratch/wenyuan/Mask_RCNN_On_Pathology/ResNet/logs/prostate20180227T2229/mask_rcnn_prostate_0040.h5


In [16]:
subset_1 = [x for x in range(513)]
dataset_test = prostate.ProstateDataset()
dataset_test.load_prostate(dataset_dir, subset_1, mode = 16)
dataset_test.prepare() # prepare the dataset class for use

In [17]:
for image_id in dataset_test.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    tumor_class_gt = 1 \
    if (sum(gt_class_id)) else 0
    
    # Run object detection
    results = model.detect([image], verbose=0)
    if (np.argmax(results[0]) != tumor_class_gt):
        print(tumor_class_gt, results, image_id / 16, image_id % 16)

1 [[ 0.94721711  0.05278282]] 143.875 14
0 [[  6.92648030e-07   9.99999285e-01]] 153.25 4
0 [[ 0.22550039  0.7744996 ]] 243.1875 3
1 [[ 0.98100233  0.0189977 ]] 277.8125 13
